In [ ]:
import chromadb
from chromadb.config import Settings

# 1. Create a Settings object with *only* the persist_directory
settings = Settings(persist_directory="./chroma_db")

# 2. Instantiate the client with those settings
client = chromadb.Client(settings=settings)

# 3. Create (or fetch) your collection as before
collection = client.get_or_create_collection(name="food_images")



In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

# 1. Use a lightweight, all-in-one model
embedder = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 2. Your sample data
docs    = [
    "Apple: 95 kcal, 0.3 g fat, 0.5 g protein",
    "Banana: 105 kcal, 0.4 g fat, 1.3 g protein"
]
metas   = [{"food": "apple"}, {"food": "banana"}]
doc_ids = ["apple_1", "banana_1"]

# 3. Embed & add to Chroma
embs = embedder.embed_documents(docs)
collection.add(
    documents=docs,
    embeddings=embs,
    metadatas=metas,
    ids=doc_ids,
)



/tmp/ipykernel_24624/4119321899.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(


In [9]:
# Query for “banana”
query = "banana"

# 1. Embed the query
query_emb = embedder.embed_query(query)

# 2. Query Chroma for the top match
result = collection.query(
    query_embeddings=[query_emb],
    n_results=1,
)

# 3. Inspect the result
print(result)


{'ids': [['banana_1']], 'embeddings': None, 'documents': [['Banana: 105 kcal, 0.4 g fat, 1.3 g protein']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'food': 'banana'}]], 'distances': [[0.8133772611618042]]}


In [11]:
from langchain.vectorstores import Chroma as LCChroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # or any other LLM

# 1. Point LangChain at your existing Chroma store
vectorstore = LCChroma(
    persist_directory="./chroma_db",
    collection_name="food_images",
    embedding_function=embedder,
)

# 2. Build a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# 3. Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),       
    retriever=retriever,
)

# 4. Ask a question
answer = qa_chain.run("How many calories are in a banana?")
print(answer)


ValidationError: 1 validation error for OpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_kwargs': {}, 'nam...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error